In [1]:
# Compatibility layer between Python 2 and Python 3
from __future__ import print_function
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import os
import sys
import pandas as pd

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras.models import load_model

Using TensorFlow backend.


In [3]:
MODEL_NAME = 'life-threatening_arrhythmias'
PATH_DATASET = '/home/nsc/seers/dataset/' + MODEL_NAME
FILE_DATASET = MODEL_NAME + '.csv'

print (os.path.join(PATH_DATASET, FILE_DATASET))

/home/nsc/seers/dataset/life-threatening_arrhythmias\life-threatening_arrhythmias.csv


In [4]:
def feature_normalize(dataset):

    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma


def show_confusion_matrix(validations, predictions):

    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap="coolwarm",
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt="d")
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()


def show_basic_dataframe_info(dataframe,
                              preview_rows=20):

    """
    This function shows basic information for the given dataframe
    Args:
        dataframe: A Pandas DataFrame expected to contain data
        preview_rows: An integer value of how many rows to preview
    Returns:
        Nothing
    """

    # Shape and how many rows and columns
    print("Number of columns in the dataframe: %i" % (dataframe.shape[1]))
    print("Number of rows in the dataframe: %i\n" % (dataframe.shape[0]))
    print("First 20 rows of the dataframe:\n")
    # Show first 20 rows
    print(dataframe.head(preview_rows))
    print("\nDescription of dataframe:\n")
    # Describe dataset like mean, min, max, etc.
    # print(dataframe.describe())

In [5]:
def read_data(path, file):

    """
    This function reads the life-threatening_arrhythmias data from a file
    Args:
        file_path: path to the CSV file
    Returns:
        A pandas dataframe
    """
    
    print(os.path.join(path, file))
    # dataset.csv
    # record	alarm_label	arrhythmia	lead_ii_max	lead_ii_min	lead_ii_file
    df_csv = pd.read_csv(os.path.join(path, file), sep=',')

    return df_csv

In [6]:
def convert_to_float(x):

    try:
        return np.float(x)
    except:
        return np.nan


# Not used right now
def feature_normalize(dataset):

    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma

In [7]:
def plot_axis(ax, x, y, title):

    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)

    
def plot_arrhythmia(path, arrhythmia, df_t, count=1):
    
    df_t = df_t[df_t['arrhythmia'] == arrhythmia]
    df_true = df_t[df_t['alarm_label'] == True].sample(frac=1).reset_index(drop=True)
    df_false = df_t[df_t['alarm_label'] == False].sample(frac=1).reset_index(drop=True)
    for idx in range(count):
        fig, (ax0, ax1) = plt.subplots(nrows=2,
             figsize=(15, 5),
             sharex=True)
        df_lead_ii = pd.read_csv(os.path.join(path, df_true['lead_ii_file'][idx]), header=None)
        df_lead_ii.columns = ['lead_ii']
        plot_axis(ax0, df_lead_ii.index, df_lead_ii['lead_ii'], df_true['lead_ii_file'][idx])
        df_lead_ii = pd.read_csv(os.path.join(path, df_false['lead_ii_file'][idx]), header=None)
        df_lead_ii.columns = ['lead_ii']
        plot_axis(ax1, df_lead_ii.index, df_lead_ii['lead_ii'], df_false['lead_ii_file'][idx])
        plt.subplots_adjust(hspace=0.2)
        fig.suptitle(arrhythmia)
        plt.subplots_adjust(top=0.90)
        plt.show()

In [8]:
def create_segments_and_labels(path, df_dataset, series_record, label_name):

    # lead-II acceleration as features
    N_FEATURES = 1
    TIME_STEPS = 2500
    
    segments = []
    labels = []
    for idx_item, value in series_record.iteritems():
        df_record = df_dataset[df.record == value]
        for idx_row, row in df_record.iterrows():
            df_lead_ii = pd.read_csv(os.path.join(path, row['lead_ii_file']), header=None)
            df_lead_ii.columns = ['lead_ii']
            
            # Normalize features for data set
            df_lead_ii['lead_ii'] = ((df_lead_ii['lead_ii'] - df['lead_ii_min']) 
                                     / (df['lead_ii_max'] - df['lead_ii_min']))
            
            # Round in order to comply to NSNumber from iOS
            df_lead_ii = df_lead_ii.round({'lead_ii': 6})
                        
            segments.append([df_lead_ii['lead_ii'].values,])
            labels.append(row[label_name])

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, TIME_STEPS, N_FEATURES)
    labels = np.asarray(labels)
    
    return reshaped_segments, labels

In [9]:
# %%

# ------- THE PROGRAM TO LOAD DATA AND TRAIN THE MODEL -------

# Set some standard parameters upfront
pd.options.display.float_format = '{:.1f}'.format
sns.set() # Default seaborn look and feel
plt.style.use('ggplot')
print('keras version ', keras.__version__)

LABELS = ["False",
          "True"]

# The number of steps within one time segment
TIME_PERIODS = 2500

# %%

keras version  2.3.1


In [10]:
print("\n--- Load, csv data ---\n")

# Load data set containing all the data from csv
df = read_data(PATH_DATASET, FILE_DATASET)


--- Load, csv data ---

/home/nsc/seers/dataset/life-threatening_arrhythmias\life-threatening_arrhythmias.csv


FileNotFoundError: [Errno 2] File b'/home/nsc/seers/dataset/life-threatening_arrhythmias\\life-threatening_arrhythmias.csv' does not exist: b'/home/nsc/seers/dataset/life-threatening_arrhythmias\\life-threatening_arrhythmias.csv'

In [ ]:
# Describe the data
show_basic_dataframe_info(df, 20)

In [ ]:
df['alarm_label'].value_counts().plot(kind='bar',
                                   title='Training Examples by Alarm Type')
plt.show()

In [ ]:
df['arrhythmia'].value_counts().plot(kind='bar',
                                  title='Training Examples by arrhythmia')
plt.show()

In [ ]:
for arrhythmia in np.unique(df["arrhythmia"]):
    plot_arrhythmia(PATH_DATASET,arrhythmia, df, count=3)

In [ ]:
# Define column name of the label vector
LABEL = "alarm_encoded"
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df[LABEL] = le.fit_transform(df["alarm_label"].values.ravel())

In [ ]:
df

In [ ]:
# %%

print("\n--- Reshape the data into segments ---\n")

# Differentiate between test set and training set
series_record = df['record']
series_record = series_record.drop_duplicates(keep='last')
#df_record = df_record.sample(frac=1).reset_index(drop=True)
series_record_train, series_record_test = train_test_split(series_record, test_size=0.1)
series_record_train = series_record_train.reset_index(drop=True)
series_record_test = series_record_test.reset_index(drop=True)


# Reshape the training data into segments
# so that they can be processed by the network
x_train, y_train = create_segments_and_labels(PATH_DATASET,
                                              df,
                                              series_record_train,
                                              LABEL)

# %%

In [ ]:
x_train

In [ ]:
print("\n--- Reshape data to be accepted by Keras ---\n")

# Inspect x data
print('x_train shape: ', x_train.shape)
# Displays (5030, 2500, 1)
print(x_train.shape[0], 'training samples')
# Displays 5030 train samples

# Inspect y data
print('y_train shape: ', y_train.shape)
# Displays (5030,)

# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]

print(x_train.shape[2])
num_classes = le.classes_.size
print(list(le.classes_))

In [ ]:
num_time_periods

In [ ]:
input_shape = (num_time_periods*num_sensors)

print('x_train shape:', x_train.shape)
# x_train shape: (5018, 2500)
print('input_shape:', input_shape)
# input_shape: (2500)

# Convert type for Keras otherwise Keras cannot process the data
x_train = x_train.astype("float32")
y_train = y_train.astype("float32")

In [ ]:
y_train

In [ ]:
# %%

# One-hot encoding of y_train labels (only execute once!)
y_train = np_utils.to_categorical(y_train, num_classes)
print('New y_train shape: ', y_train.shape)
# (4173, 6)

# %%

In [ ]:
y_train

In [ ]:
TIME_PERIODS

In [ ]:
print("\n--- Create neural network model ---\n")

# 1D CNN neural network
model_m = Sequential()
#model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
model_m.add(Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
model_m.add(Conv1D(100, 10, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(GlobalAveragePooling1D())
model_m.add(Dropout(0.5))
model_m.add(Dense(num_classes, activation='softmax'))
print(model_m.summary())
# Accuracy on training data: 99%
# Accuracy on test data: 91%

In [ ]:
print("\n--- Fit the model ---\n")

# The EarlyStopping callback monitors training accuracy:
# if it fails to improve for two consecutive epochs,
# training stops early
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=10)
]

model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

# Hyper-parameters
BATCH_SIZE = 250
EPOCHS = 100

# Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
history = model_m.fit(x_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

In [ ]:
# %%

print("\n--- Learning curve of model training ---\n")

# summarize history for accuracy and loss
plt.figure(figsize=(6, 4))
plt.plot(history.history['acc'], "g--", label="Accuracy of training data")
plt.plot(history.history['val_acc'], "g", label="Accuracy of validation data")
plt.plot(history.history['loss'], "r--", label="Loss of training data")
plt.plot(history.history['val_loss'], "r", label="Loss of validation data")
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

#%%

In [ ]:
print("\n--- Check against test data ---\n")

x_test, y_test = create_segments_and_labels(PATH_DATASET,
                                              df,
                                              series_record_test,
                                              LABEL)

In [ ]:
x_test.shape

In [ ]:
x_test = x_test.astype("float32")
y_test = y_test.astype("float32")

y_test = np_utils.to_categorical(y_test, num_classes)

score = model_m.evaluate(x_test, y_test, verbose=1)

print("\nAccuracy on test data: %0.2f" % score[1])
print("\nLoss on test data: %0.2f" % score[0])

In [ ]:
# %%

print("\n--- Confusion matrix for test data ---\n")

y_pred_test = model_m.predict(x_test)
# Take the class with the highest probability from the test predictions
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = np.argmax(y_test, axis=1)

show_confusion_matrix(max_y_test, max_y_pred_test)

# %%

print("\n--- Classification report for test data ---\n")

print(classification_report(max_y_test, max_y_pred_test))

In [ ]:
model_m.save(MODEL_NAME+'.h5')

In [ ]:
model_l = load_model(MODEL_NAME+'.h5')

In [ ]:
score_l = model_l.evaluate(x_test, y_test, verbose=1)

print("\nAccuracy on test data: %0.2f" % score_l[1])
print("\nLoss on test data: %0.2f" % score_l[0])